In [1]:
import warnings
warnings.filterwarnings("ignore")

!pip install yargy

In [2]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

In [3]:
Name = fact(
    'Name',
    ['first', 'last'],
)
Person = fact(
    'Person',
    ['position', 'name']
)

LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
)
FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
)

POSITION = morph_pipeline([
    'разработчик',
    'программист'
])

gnc = gnc_relation()
NAME = rule(
    FIRST.interpretation(
        Name.first
    ).match(gnc),
    LAST.interpretation(
        Name.last
    ).match(gnc)
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position
    ).match(gnc),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)

In [4]:
parser = Parser(PERSON)

In [5]:
match = parser.match('разработчик Дмитрий Еськов')
print(match)

Match(tokens=[MorphToken(value='разработчик', span=[0, 11), type='RU', forms=[Form('разработчик', Grams(NOUN,anim,masc,nomn,sing))]), MorphToken(value='Дмитрий', span=[12, 19), type='RU', forms=[Form('дмитрий', Grams(NOUN,Name,anim,masc,nomn,sing))]), MorphToken(value='Еськов', span=[20, 26), type='RU', forms=[Form('еськов', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])], span=[0, 26))


In [6]:
with open('20150618muchoojo.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['разработчик', 'Дмитрий', 'Еськов']
['программист', 'Николай', 'Тарасенко']


In [8]:
from ipymarkup import show_span_box_markup as show_markup

In [9]:
matches = parser.findall(text)
spans = [_.span for _ in matches]
spans

[[8, 34), [122, 151)]

In [10]:
text[:]

'Молодой разработчик Дмитрий Еськов написал свою книгу по гейм дизайну и стал учителем для многих программистов. Например, программист Николай Тарасенко, используя труды Еськова создал один из популярнейших продуктов на Российском рынке.'

In [11]:
show_markup(text, spans)